In [366]:
from datetime import datetime

floorsheet_url  = "http://www.nepalstock.com/main/floorsheet/index/page/?contract-no=&stock-symbol=&buyer=&seller=&_limit=20000"
floorsheet_columns = ["Contact No", "Stock Symbol", "Buyer Broker", "Seller Broker", "Quantity", "Rate", "Amount"] 
df = pd.DataFrame(columns = floorsheet_columns)
pages = 3

for i in range(pages):
    url = floorsheet_url.replace("page", str(i+1))
    data = pd.read_html(url)[0]
    data = data.iloc[2:-3, 1:-2]    
    data.columns = floorsheet_columns
    df = df.append(data)    
df.set_index("Contact No")

data = df.copy()
data = data.apply(lambda row: pd.Series([int(row["Contact No"]), row["Stock Symbol"], 
                        row["Buyer Broker"], row["Seller Broker"],
                        int(row["Quantity"]), float(row["Rate"]),
                        float(row["Amount"])
                       ]), axis = 1)
data.columns = floorsheet_columns
data.set_index("Contact No")

pd.set_option('display.float_format', lambda x: '%.1f' % x)
trade_data  = data.copy()
trade_data.pop("Contact No")
trade_data.pop("Rate")

buy_stat = trade_data.groupby(["Buyer Broker", "Stock Symbol"]).sum()
sell_stat = trade_data.groupby(["Seller Broker", "Stock Symbol"]).sum()

buy_stat.index.names = ["Broker", "Stock Symbol"]
sell_stat.index.names = ["Broker", "Stock Symbol"]
trade_stat = buy_stat.sub(sell_stat, fill_value = 0)
trade_stat["Average Rate"] = trade_stat["Amount"]/trade_stat["Quantity"]
trade_stat.to_csv(datetime.today().strftime('%Y-%m-%d') + ".csv")